## Training
### Import library

In [1]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split
from patchify import patchify
from unetr_2d import build_unetr_2d
import matplotlib.pyplot as plt

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
np.random.seed(42)
tf.random.set_seed(42)

### Configuration

In [4]:
# UNETR configuration
cf = {}
cf["image_size"] = 256
cf["num_classes"] = 11
cf["num_channels"] = 3
cf["num_layers"] = 12
cf["hidden_dim"] = 128
cf["mlp_dim"] = 32
cf["num_heads"] = 6
cf["dropout_rate"] = 0.1
cf["patch_size"] = 16
cf["num_patches"] = (cf["image_size"]**2) // (cf["patch_size"]**2)
cf["flat_patches_shape"] = (
    cf["num_patches"],
    cf["patch_size"] * cf["patch_size"] * cf["num_channels"]
)

### Input pipeline

In [20]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def load_dataset(path):
    train_x = sorted(glob(os.path.join(path, "lapa_train", "images", "*.jpg")))
    train_y = sorted(glob(os.path.join(path, "lapa_train", "labels", "*.png")))

    valid_x = sorted(glob(os.path.join(path, "lapa_val", "images", "*.jpg")))
    valid_y = sorted(glob(os.path.join(path, "lapa_val", "labels", "*.png")))

    test_x = sorted(glob(os.path.join(path, "lapa_test", "images", "*.jpg")))
    test_y = sorted(glob(os.path.join(path, "lapa_test", "labels", "*.png")))

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image(path):
    path = path.decode()
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (cf["image_size"], cf["image_size"]))
    image = image / 255.0

    # Processing to patches
    patch_shape = (cf["patch_size"], cf["patch_size"], cf["num_channels"])
    patches = patchify(image, patch_shape, cf["patch_size"])
    patches = np.reshape(patches, cf["flat_patches_shape"])
    patches = patches.astype(np.float32)

    return patches

def read_mask(path):
    path = path.decode()
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (cf["image_size"], cf["image_size"]))
    mask = mask.astype(np.int32)
    return mask

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        y = tf.one_hot(y, cf["num_classes"])
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape(cf["flat_patches_shape"])
    y.set_shape([cf["image_size"], cf["image_size"], cf["num_classes"]])
    return x, y
        
def tf_dataset(X, Y, batch=1):
    ds = tf.data.Dataset.from_tensor_slices((X, Y))
    ds = ds.map(tf_parse).batch(batch).prefetch(10)
    return ds

In [21]:
# Seeding
np.random.seed(42)
tf.random.set_seed(42)

# Directory for storing files
create_dir("files")

# Hyperparameters
batch_size = 16
lr = 0.1
num_epochs = 1
model_path = os.path.join("files", "model.keras")
csv_path = os.path.join("files", "log.csv")

# RGB Code and Classes
rgb_codes = [
    [0, 0, 0], [0, 153, 255], [102, 255, 153], [0, 204, 153],
    [255, 255, 102], [255, 255, 204], [255, 153, 0], [255, 102, 255],
    [102, 0, 51], [255, 204, 255], [255, 0, 102]
]

classes = [
    "background", "skin", "left eyebrow", "right eyebrow",
    "left eye", "right eye", "nose", "upper lip", "inner mouth",
    "lower lip", "hair"
]

dataset_path = "D:\Documents\GitHub\Triple-View-R-Net\data\LaPa"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)

print(f"Train: \t{len(train_x)} - {len(train_y)}")
print(f"Valid: \t{len(valid_x)} - {len(valid_y)}")
print(f"Test: \t{len(test_x)} - {len(test_y)}")

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

Train: 	18168 - 18168
Valid: 	2000 - 2000
Test: 	2000 - 2000


In [22]:
print(train_dataset)
print(valid_dataset)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 768), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 11), dtype=tf.float32, name=None))>
<PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 768), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 11), dtype=tf.float32, name=None))>


In [25]:
for image, mask in train_dataset.batch(1).take(1):
    print(image.shape, mask.shape)

(1, 16, 256, 768) (1, 16, 256, 256, 11)


### Build Model

In [17]:
model = build_unetr_2d(cf)
print(model.summary())
model.compile(
    loss="categorical_crossentropy", 
    optimizer=SGD(lr)
)

Model: "UNETR_2D"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 768)]   0           []                               
                                                                                                  
 dense_25 (Dense)               (None, 256, 128)     98432       ['input_2[0][0]']                
                                                                                                  
 tf.__operators__.add_1 (TFOpLa  (None, 256, 128)    0           ['dense_25[0][0]']               
 mbda)                                                                                            
                                                                                                  
 layer_normalization_24 (LayerN  (None, 256, 128)    256         ['tf.__operators__.add_1[0

### Training Loop

In [ ]:
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
]

history = model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=callbacks
)

## Testing

In [60]:
def grayscale_to_rgb(mask, rgb_codes):
    h, w = mask.shape[0], mask.shape[1]
    mask = mask.astype(np.int32)
    output = []

    for i, pixel in enumerate(mask.flatten()):
        output.append(rgb_codes[pixel])

    output = np.reshape(output, (h, w, 3))
    return output

def save_results(image_x, mask, pred, save_image_path):
    mask = np.expand_dims(mask, axis=-1)
    mask = grayscale_to_rgb(mask, rgb_codes)

    pred = np.expand_dims(pred, axis=-1)
    pred = grayscale_to_rgb(pred, rgb_codes)

    cat_images = np.concatenate([image_x, mask, pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

In [ ]:
# Load the model 
model_path = os.path.join("files", "model.keras")
model = tf.keras.models.load_model(model_path)

In [69]:
# Directory for storing files
create_dir(f"results")

# Prediction
for index, (x, y) in enumerate(tqdm(zip(test_x, test_y), total=10)):
    
    # Just first 10 testing images
    if (index==10): break
    
    # Extracting the name 
    name = x.split("/")[-1].split(".")[0]

    # Read the image
    image = cv2.imread(x, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (cf["image_size"], cf["image_size"]))
    x = image

    patch_shape = (cf["patch_size"], cf["patch_size"], cf["num_channels"])
    patches = patchify(x / 255.0, patch_shape, cf["patch_size"])
    patches = np.reshape(patches, cf["flat_patches_shape"])
    patches = patches.astype(np.float32)
    patches = np.expand_dims(patches, axis=0)

    # Read Mask
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (cf["image_size"], cf["image_size"]))
    mask = mask.astype(np.int32)

    # Prediction
    pred = model.predict(patches, verbose=0)[0]
    pred = np.argmax(pred, axis=-1)
    pred = pred.astype(np.int32)

    # Save the results
    save_image_path = f"D:/Documents/GitHub/Triple-View-R-Net/UNETR_LaPa/results/test_{index}.png"
    save_results(image, mask, pred, save_image_path)

100%|██████████| 10/10 [00:03<00:00,  2.66it/s]
